In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# # model_name = '/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'
# # model_name = "/kaggle/input/mistral-7b-instruct-v0.2-gptq/transformers/1/1/Mistral-7B-Instruct-v0.2-GPTQ"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     trust_remote_code=True,
# )
model_id = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
prompt_format = """
# Instruction
Your task is to predict the topic from the list of the given text:
1. Car-free cities
2. Does the electoral college work
3. A Cowboy Who Rode the Waves
4. Exploring Venus
5. Driverless cars
6. Facial action coding system
7. The Face on Mars
8. None of the above

# Input
The text is: 
```text
{text}
```

# Output
Return JSON object only. Dont GIVE Any Explaination!!
```json
{{"match_number": }}
```
""".strip()
prompt = prompt_format.format(text="""People always wish they had the same technology that they have seen in movies, or the best new piece of technology that is all over social media. However, nobody seems to think of the risks that these kinds of new technologies may have. Cars have been around for many decades, and now manufacturers are starting to get on the bandwagon and come up with the new and improved technology that they hope will appeal to everyone. As of right now, it seems as though the negative characteristics of these cars consume the positive idea that these manufacturers have tried to convey.\n\nCurrently, this new technology in cars has a very long way to go before being completely "driverless".""")

In [ ]:
# messages = [
#     {"role": "user", "content": prompt}
# ]

# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(model.device)

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# outputs = model.generate(
#     input_ids,
#     max_new_tokens=256,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9,
# )
# response = outputs[0][input_ids.shape[-1]:]
# print(tokenizer.decode(response, skip_special_tokens=True))

In [ ]:
def chat_for_complation(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
#     encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
# #     text = f"""
# #     <s>[INST] <<SYS>>
# # {prompt}
# # <<SYS>> [/INST]
# #     """.strip()
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    outputs = model.generate(
        input_ids,
        max_new_tokens=32,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

#     encodeds = tokenizer(text, add_special_tokens=False, return_tensors="pt")
#     model_inputs = encodeds
#     generated_ids = model.generate(input_ids, max_new_tokens=128, do_sample=True, temperature=0.1)
#     decoded = tokenizer.batch_decode(generated_ids)[0]
#     return decoded[decoded.index("[/INST]"):][len("[/INST]"):-len("</s>")].strip()
#     return decoded

In [ ]:
import json

prompt_format = """
# Instruction
Your task is to predict the topic from the list of the given text:
1. Car-free cities
2. Does the electoral college work
3. A Cowboy Who Rode the Waves
4. Exploring Venus
5. Driverless cars
6. Facial action coding system
7. The Face on Mars
8. None of the above

# Input
The text is: 
```text
{text}
```

# Output
Return JSON object only.
```json
{{"match_number": }}
```
""".strip()


def chat_for_match(full_text):
    text = " ".join(full_text.strip().split()[:128])
    prompt = prompt_format.format(text=text)
    completion = chat_for_complation(prompt)
    if "{" not in completion or "}" not in completion:
        return -1
    try:
        data = json.loads(completion[completion.find("{"): completion.find("}") + 1])
    except Exception as e:
        return -1
    if "match_number" not in data:
        return -1
    try:
        return int(data["match_number"])
    except Exception:
        return -1

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()


DEBUG = False
if DEBUG:
    df = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv")
    df = df.sample(100)
else:
    df = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
    df = df.sample(min(1200, len(df)))

df["topic"] = df["full_text"].progress_apply(chat_for_match)
# print(df.shape[0], df["matched"].sum())
df["topic"].value_counts()

In [ ]:
d = df["topic"].value_counts()
print(dict(zip(d.index, d.values)))
other_type = dict(zip(d.index, d.values)).get(8, 0)
if other_type > 100:
    raise ValueError("many other prompt!")
else:
    sub = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv")
    sub.to_csv("submission.csv", index=False)